In [1]:
# all import statements needed for the project, for example:
import math
import bs4
import requests
import sqlalchemy as db
import pandas as pd 
import numpy as np
import datetime as dt
import statsmodels.api as sm #统计
from statsmodels.tsa.stattools import adfuller #ADF检验
import matplotlib as mpl #画图
import matplotlib.pyplot as plt
mpl.rcParams['font.family']='serif'
plt.style.use('seaborn') 
import os #地址
os.chdir('/Users/yw/Desktop/4501 Project') 

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 2: Storing Cleaned Data

1. use SQLAlchemy, create a SQLite database to load in preprocessed datasets;
2. create and populate 4 tables: one for sampled datasets of Yellow Taxi trips, one for Uber trips, one for hourly weather information, and one for daily weather information. 
3. create a schema.sql file that defines each table’s schema. 

In [185]:
engine = db.create_engine(DATABASE_URL)

In [186]:
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY,
    date TEXT,
    dailysustainedwindspeed FLOAT,
    hourlyprecipitation FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY,
    date TEXT,
    dailyaveragewindspeed FLOAT,
    dailypeakwindspeed FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi (
    id INTEGER PRIMARY KEY,
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
    month INTEGER,
    pickup_latitude FLOAT, 
    pickup_longitude FLOAT,
    dropoff_latitude FLOAT,
    dropoff_longitude FLOAT,
    distance FLOAT,
    tip FLOAT,
    passenger_count INTEGER
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber (
    id INTEGER PRIMARY KEY,
    fare_amount FLOAT,
    pickup_datetime TEXT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,  
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    passenger_count INTEGER,
    date TEXT,
    week INTEGER,
    distance FLOAT
);
"""
 

In [187]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [188]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

1. define function to insert data from dataframe to SQL database
2. use dictionary to store data and data name
3. add data to database

In [189]:
def write_dataframes_to_table(table_to_df_dict):
    for k,v in map_table_name_to_dataframe.items():
        v.to_sql(k,con=engine,if_exists='append',index_label='id')
    #hour.to_sql('hourly_weather',con=engine,if_exists='append',index_label='id')
    #daily.to_sql('daily_weather',con=engine,if_exists='append',index_label='id')
    #uber_data.to_sql('uber',con=engine,if_exists='append',index_label='id')   

In [190]:
map_table_name_to_dataframe = {
    "taxi": taxi_data,
    "uber": uber_data,
    "hourly_weather": hour,
    "daily_weather": daily,
}

In [191]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [√] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [√] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [√] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [192]:
def write_query_to_file(query, outfile):
    with open(outfile, 'w') as file:
        file.write(query)

### Query N

1. define a SQL query for each of the following questions - one query per question. 
2. Save each query as a .sql file, naming it something illustrative of what the query is for, e.g. top_10_hottest_days.sql.

$Question \ 2$

$\text{From 01-2009 to 06-2015, what day of the week was the most popular to take an uber? The result should have 7 bins.}$

In [193]:
QUERY_2 = """
SELECT week, COUNT(week) FROM (SELECT DISTINCT week,date FROM uber
WHERE date BETWEEN "2009-01-01" AND "2015-07-01" )
GROUP BY week
ORDER BY COUNT(week) DESC
"""

In [194]:
engine.execute(QUERY_2).fetchall()

[(7, 339), (6, 339), (5, 339), (4, 339), (2, 339), (1, 339), (3, 338)]

In [195]:
write_query_to_file(QUERY_2, "popular_day_of_week_uber.sql")

$ Question \ 3$

$\text{What is the 95% percentile of distance traveled for all hired trips during July 2013?}$

In [196]:
QUERY_3="""
WITH two AS 
(SELECT pickup_datetime,distance FROM uber
WHERE "2013-06-30T00:00"< pickup_datetime AND pickup_datetime < '2013-07-31T00:00:00'
UNION ALL
SELECT pickup_datetime,distance FROM taxi
WHERE "2013-06-30T00:00"<pickup_datetime AND pickup_datetime < '2013-07-31T00:00:00')

SELECT distance AS '95%distance'FROM two
ORDER BY distance 
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM two) * 95 / 100 - 1 ;
"""

In [197]:
engine.execute(QUERY_3).fetchall()

[(10.234680206341519,)]

In [198]:
write_query_to_file(QUERY_3, "95%percentile_distance.sql")

$Question \ 5$

$\text{Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?}$

In [201]:
#peak wind speed
QUERY_5 = """
WITH ride AS
(SELECT DATE(pickup_datetime) AS date, count(pickup_datetime) AS count from (SELECT pickup_datetime FROM uber
WHERE "2013-12-31T00:00"< pickup_datetime AND pickup_datetime < '2014-12-31T00:00:00'
UNION ALL
SELECT pickup_datetime FROM taxi
WHERE "2013-12-31T00:00"<pickup_datetime AND pickup_datetime < '2014-12-31T00:00:00')
GROUP BY DATE(pickup_datetime))

SELECT * FROM(SELECT DATE(daily_weather.date) AS date, daily_weather.dailyaveragewindspeed, 
daily_weather.dailypeakwindspeed, ride.count
FROM daily_weather 
LEFT OUTER JOIN ride 
ON DATE(daily_weather.date)=DATE(ride.date)
ORDER BY daily_weather.dailypeakwindspeed DESC
LIMIT 10
)
"""

In [202]:
engine.execute(QUERY_5).fetchall()

[('2014-03-13', 14.1, 40.0, 106),
 ('2014-12-07', 11.8, 40.0, 73),
 ('2014-02-13', 12.6, 39.0, 63),
 ('2014-02-27', 8.4, 39.0, 102),
 ('2014-01-07', 13.1, 38.0, 77),
 ('2014-02-14', 10.4, 38.0, 71),
 ('2014-03-26', 11.9, 38.0, 96),
 ('2014-04-05', 8.4, 36.0, 92),
 ('2014-04-23', 9.4, 36.0, 75),
 ('2014-11-02', 10.8, 36.0, 80)]

In [203]:
#average wind speed
QUERY_5 = """
WITH ride AS
(SELECT DATE(pickup_datetime) AS date, count(pickup_datetime) AS count from (SELECT pickup_datetime FROM uber
WHERE "2013-12-31T00:00"< pickup_datetime AND pickup_datetime < '2014-12-31T00:00:00'
UNION ALL
SELECT pickup_datetime FROM taxi
WHERE "2013-12-31T00:00"<pickup_datetime AND pickup_datetime < '2014-12-31T00:00:00')
GROUP BY DATE(pickup_datetime))

SELECT * FROM(SELECT DATE(daily_weather.date) AS date, daily_weather.dailyaveragewindspeed, 
daily_weather.dailypeakwindspeed, ride.count
FROM daily_weather 
LEFT OUTER JOIN ride 
ON DATE(daily_weather.date)=DATE(ride.date)
ORDER BY daily_weather.dailyaveragewindspeed DESC
LIMIT 10
)
"""

In [204]:
engine.execute(QUERY_5).fetchall()

[('2014-03-13', 14.1, 40.0, 106),
 ('2014-01-07', 13.1, 38.0, 77),
 ('2014-02-13', 12.6, 39.0, 63),
 ('2014-01-02', 12.2, 28.0, 63),
 ('2014-03-26', 11.9, 38.0, 96),
 ('2014-12-07', 11.8, 40.0, 73),
 ('2014-12-08', 11.5, 29.0, 77),
 ('2014-03-29', 10.8, 35.0, 107),
 ('2014-11-02', 10.8, 36.0, 80),
 ('2014-01-03', 10.4, 29.0, 41)]

In [205]:
write_query_to_file(QUERY_5, "top10_windest_hiredtrips.sql")